In [1]:
from pathlib import Path
import numpy as np
from pandas import read_csv, DataFrame, get_dummies
from tqdm import tqdm

from tensorflow import convert_to_tensor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv2D, BatchNormalization, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.layers.core import Dropout
from tensorflow.keras import Model

In [2]:
sample_csv = Path("/content/drive/MyDrive/DATASET/ORIGINAL/Address/csv/a1 (1).csv")
DB_ROOT = "/content/drive/MyDrive/DATASET/ORIGINAL"

sample_csv.exists()

True

## CONFIG

In [3]:
class CSVVideoDatasetGenerator():

    def __init__(self, database_root, video_frames=50, video_input_features=1629):
        self.db_root = Path(database_root)
        self.video_frames = video_frames
        self.video_input_features = video_input_features
        self.get_summary()


    def start(self, kls_select="*", randomize=True, save=False, save_dir=".", limit_per_class=None):
        kls_slice = self.__get_slice(kls_select)

        if save:
            save_dir = Path(save_dir)
            if not save_dir.exists(): raise ValueError("save dir does not exist")

        x = None
        y = []
        for kls in kls_slice:
            with tqdm(total=self.summary['detected'][kls], desc=f"Generating {kls}", initial=0, unit_scale=True) as pbar:
                for csv_path in (self.db_root / kls / 'csv').glob('*.csv'):
                    csv_df = self.read_csv_as_df(csv_path, self.video_input_features)
                    values = np.expand_dims(csv_read_df.values, axis=0)
                    if isinstance(x, type(None)):
                        x = values
                    else:
                        x = np.vstack((x, values))

                    y.append(kls)

                    pbar.update(1)

        y  = get_dummies(DataFrame(y)).values

        if randomize:
            p = np.random.permutation(y.shape[0])
            y = y[p]
            x = x[p]

        if save:
            np.save(str(save_dir / 'x2.npy'), x)
            np.save(str(save_dir / 'y2.npy'), y)

        return x, y

    def __get_slice(self, kls_select):
        if isinstance(kls_select, str):
            if kls_select == "*":
                return self.summary['classes valid']
        elif isinstance(kls_select, list):
            return [kls for kls in self.summary['classes valid'] if kls in kls_select]
                

    def get_summary(self, recalculate=True):
        if recalculate:
            summary = {}
            summary['classes detected'] = [x.name for x in self.db_root.glob('*') if x.is_dir]
            summary['classes invalid'] = [kls for kls in summary['classes detected'] if not (self.db_root / kls / 'csv').exists()]
            summary['classes valid'] = [kls for kls in summary['classes detected'] if kls not in summary['classes invalid']]

            summary['detected'] = {}
            for kls in summary['classes valid']:
                summary['detected'][kls] = len([x for x in (self.db_root / kls / 'csv').glob('*.csv') if x.is_file()])

            self.summary = summary
        else:
            if not self.summary: return None

        return self.summary

    # utility
    def read_csv_as_df(self, csv_path, video_input_features) -> DataFrame:
        return read_csv(str(csv_path), names=list(range(0 , video_input_features)))

dg = CSVVideoDatasetGenerator(DB_ROOT)
# SUMMARY = dg.get_summary()

In [4]:
# on save data will overwrite
# X and Y gets overwritten THEREFORE FOR SAFETY IT IS COMMENTED OUT
# ABSOLUTELY make SURE DIR IS EMPTY BEFORE RUNNING THIS
X, Y = dg.start(kls_select=["Address", 'Work'], save=True, save_dir="/content/drive/MyDrive/CACHE")

Generating Address:   0%|          | 0.00/300 [00:00<?, ?it/s]


NameError: ignored

In [ ]:
X = np.load("/content/drive/MyDrive/CACHE/x2.npy")
Y = np.load("/content/drive/MyDrive/CACHE/y2.npy")
X.shape, Y.shape

((542, 50, 1629), (542, 2))

In [ ]:
# DROP FACE
TOTAL_FACE_COORDS = 468 * 3
FROM = 0
TO = TOTAL_FACE_COORDS
FX = X[:, :, TO:].copy()
print(FX.shape)
FX = np.expand_dims(FX, axis=3)
FX.shape

(542, 50, 225)


(542, 50, 225, 1)

In [ ]:
a = np.array([[1, 1], [2, 2], [3, 3], [4, 4]])
b = np.array([1, 2, 3, 4])
a.shape, b.shape
p = np.random.permutation(4)
a[p], b[p]
dg.get_summary(False)
list((Path(DB_ROOT) / 'Address' / 'csv').glob('*.csv'))[0].name
X[0][0], Y

(array([ 0.49330738,  0.36236238, -0.01819375, ...,  0.3727324 ,
         2.6617105 , -0.17472517]), array([[0, 1],
        [1, 0],
        [1, 0],
        ...,
        [0, 1],
        [1, 0],
        [1, 0]], dtype=uint8))

In [ ]:
class stop_(Callback): 
    ACCURACY_THRESHOLD = 0.95
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') > self.ACCURACY_THRESHOLD):
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(self.ACCURACY_THRESHOLD*100))   
            self.model.stop_training = True

class stop_categorical_(Callback): 
    ACCURACY_THRESHOLD = 0.95
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('categorical_accuracy') > self.ACCURACY_THRESHOLD):
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(self.ACCURACY_THRESHOLD*100))   
            self.model.stop_training = True

class save_(Callback):
    count = 1
    after = 10
    best = 0
    best_step = 10
    best_step_till = 90
    save_path = "/content/drive/MyDrive/MODEL/Model_300"
    def on_epoch_end(self, epoch, logs={}): 

        if(logs.get('accuracy') < self.best_step_till):
            if logs.get('accuracy') > self.best:
                self.best = logs.get('accuracy')
                self.model.save(str(self.save_path))
        else:
            if not (logs.get('accuracy') % self.after) and (logs.get('accuracy')>self.best):
                self.model.save(str(self.save_path))
                diff = abs(self.best - logs.get('accuracy'))
                if diff > self.best_step:
                    self.best += (diff // 10)*self.best_step+self.best_step
                else:

                    self.best += self.best_step
class save_categorical_(Callback):
    count = 1
    after = 10
    best = 0
    best_step = 10
    best_step_till = 90
    save_path = "/content/drive/MyDrive/MODEL/Model_300"
    def on_epoch_end(self, epoch, logs={}): 

        if(logs.get('categorical_accuracy') < self.best_step_till):
            if logs.get('categorical_accuracy') > self.best:
                self.best = logs.get('categorical_accuracy')
                self.model.save(str(self.save_path))
        else:
            if not (logs.get('categorical_accuracy') % self.after) and (logs.get('categorical_accuracy')>self.best):
                self.model.save(str(self.save_path))
                diff = abs(self.best - logs.get('categorical_accuracy'))
                if diff > self.best_step:
                    self.best += (diff // 10)*self.best_step+self.best_step
                else:
                    self.best += self.best_step



def lstm_model_compiled(input_shape, out_shape):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=input_shape))
    model.add(LSTM(256, return_sequences=True, activation='relu'))
    model.add(LSTM(128, return_sequences=False, activation='relu'))
    
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(out_shape, activation='softmax'))

    a = Adam(learning_rate=0.001)
    H = model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model, H

def lstm_model_massive_compiled(input_shape, out_shape):
    model = Sequential()
    model.add(LSTM(256, return_sequences=True, activation='relu', input_shape=input_shape))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(256, return_sequences=False, activation='relu'))
    
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(out_shape, activation='softmax'))

    a = Adam(learning_rate=0.001)
    H = model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model, H

def lstm_model_overly_massive_compiled(input_shape, out_shape):
    model = Sequential()
    model.add(LSTM(2468, return_sequences=True, activation='relu', input_shape=input_shape))
    model.add(LSTM(1024, return_sequences=True, activation='relu'))
    model.add(LSTM(512, return_sequences=True, activation='relu'))
    model.add(LSTM(256, return_sequences=False, activation='relu'))
    
    model.add(Dense(2468, activation='relu'))
    model.add(Dense(1025, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(out_shape, activation='softmax'))

    a = Adam(learning_rate=0.001)
    H = model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model, H

def conv2d_model(input_shape, out_shape):
    model = Sequential()
    model.add(Conv2D(128, (2, 9), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((3, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    # model.add(Conv2D(32, (3, 3), activation='relu'))
    # model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    # model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(out_shape, activation='softmax'))

    a = Adam(learning_rate=0.001)
    H = model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model, H

class KerasModelTrainer():
    def __init__(self, model=None):
        self.set_compiled_model(model)

    def set_compiled_model(self, model):
        if not model:
            self.model = None
        else:
            if not isinstance(model, Model): raise ValueError('Model not keras model')
            self.model = model

    class ModelNotSet(Exception): pass
    def start_training(self, x, y, epochs, callbacks=[], verbose=1, validation_split=0.2, batch_size=32):
        if not self.model: raise self.ModelNotSet("Model not set use set_compiled_model first")

        return self.model.fit(x, y, epochs=epochs, verbose=verbose, callbacks=callbacks, validation_split=validation_split)

In [ ]:
model_complied, H = conv2d_model(FX.shape[1:], Y.shape[1])
KT = KerasModelTrainer(model_complied)
stop_training = stop_()
stop_training_categorical = stop_categorical_()
save_model = save_()
save_model_categorical = save_categorical_()

In [ ]:
model_complied.summary() # conv2d model # FACE dROPPED # rectangular filter # 2 classes only
Hist = KT.start_training(FX, Y, 1000, callbacks=[stop_training_categorical, save_model_categorical], validation_split=0.2)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 49, 217, 128)      2432      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 72, 128)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 14, 70, 64)        73792     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 7, 35, 64)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 15680)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               2007168   
_________________________________________________________________
dense_17 (Dense)             (None, 64)               

KeyboardInterrupt: ignored

In [ ]:
model_complied.summary() # conv2d model # FACE dROPPED # rectangular filter
Hist = KT.start_training(FX, Y, 1000, callbacks=[stop_training_categorical, save_model_categorical], validation_split=0.2)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 49, 217, 128)      2432      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 72, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 70, 64)        73792     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 7, 35, 64)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 15680)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               2007168   
_________________________________________________________________
dense_14 (Dense)             (None, 64)               

KeyboardInterrupt: ignored

In [ ]:
model_complied.summary() # conv2d model # FACE dROPPED #
Hist = KT.start_training(FX, Y, 1000, callbacks=[stop_training_categorical, save_model_categorical], validation_split=0.2)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 44, 219, 128)      6400      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 73, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 12, 71, 64)        73792     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 35, 64)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 13440)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               1720448   
_________________________________________________________________
dense_11 (Dense)             (None, 64)               

KeyboardInterrupt: ignored

In [ ]:
model_complied.summary() # conv2d model # FACE dROPPED #
Hist = KT.start_training(FX, Y, 1000, callbacks=[stop_training_categorical, save_model_categorical], validation_split=0.2)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 223, 128)      1280      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 111, 128)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 109, 64)       73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 54, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 52, 32)         18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 26, 32)         0         
_________________________________________________________________
flatten (Flatten)            (None, 3328)             

KeyboardInterrupt: ignored

In [ ]:
model_complied.summary() # model lstm overly massive # FACE dROPPED # Adam https://www.youtube.com/watch?v=doDUihpj6ro&t=3212s
Hist = KT.start_training(FX, Y, 1000, callbacks=[stop_training_categorical, save_model_categorical], validation_split=0.2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50, 2468)          26595168  
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 1024)          14307328  
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 512)           3147776   
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense (Dense)                (None, 2468)              634276    
_________________________________________________________________
dense_1 (Dense)              (None, 1025)              2530725   
_________________________________________________________________
dense_2 (Dense)              (None, 512)               5

KeyboardInterrupt: ignored

In [ ]:
model_complied.summary() # model lstm massive # FACE dROPPED # Adam https://www.youtube.com/watch?v=doDUihpj6ro&t=3212s
Hist = KT.start_training(FX, Y, 1000, callbacks=[stop_training_categorical, save_model_categorical], validation_split=0.2)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_45 (LSTM)               (None, 50, 256)           493568    
_________________________________________________________________
lstm_46 (LSTM)               (None, 50, 128)           197120    
_________________________________________________________________
lstm_47 (LSTM)               (None, 50, 128)           131584    
_________________________________________________________________
lstm_48 (LSTM)               (None, 256)               394240    
_________________________________________________________________
dense_83 (Dense)             (None, 1024)              263168    
_________________________________________________________________
dense_84 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_85 (Dense)             (None, 256)             

KeyboardInterrupt: ignored

In [ ]:
model_complied.summary() # FACE dROPPED # Adam https://www.youtube.com/watch?v=doDUihpj6ro&t=3212s
Hist = KT.start_training(FX, Y, 1000, callbacks=[stop_training_categorical, save_model_categorical], validation_split=0.2)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_42 (LSTM)               (None, 50, 128)           181248    
_________________________________________________________________
lstm_43 (LSTM)               (None, 50, 256)           394240    
_________________________________________________________________
lstm_44 (LSTM)               (None, 128)               197120    
_________________________________________________________________
dense_78 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_79 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_80 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_81 (Dense)             (None, 32)              

KeyboardInterrupt: ignored

In [ ]:
model_complied.summary() # Adam https://www.youtube.com/watch?v=doDUihpj6ro&t=3212s
KT.start_training(X, Y, 1000, callbacks=[stop_training_categorical, save_model_categorical], validation_split=0.2)

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_39 (LSTM)               (None, 50, 128)           900096    
_________________________________________________________________
lstm_40 (LSTM)               (None, 50, 256)           394240    
_________________________________________________________________
lstm_41 (LSTM)               (None, 128)               197120    
_________________________________________________________________
dense_73 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_74 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_75 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_76 (Dense)             (None, 32)              

KeyboardInterrupt: ignored

In [ ]:
model_complied.summary() # RMSPROP default
KT.start_training(X, Y, 1000, callbacks=[stop_training, save_model], validation_split=0.2)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 50, 64)            433664    
_________________________________________________________________
lstm_22 (LSTM)               (None, 50, 128)           98816     
_________________________________________________________________
lstm_23 (LSTM)               (None, 256)               394240    
_________________________________________________________________
dense_43 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_44 (Dense)             (None, 124)               15996     
_________________________________________________________________
dense_45 (Dense)             (None, 64)                8000      
_________________________________________________________________
dense_46 (Dense)             (None, 32)              

KeyboardInterrupt: ignored

In [ ]:
model_complied.summary() # sgd default
KT.start_training(X, Y, 1000, callbacks=[stop_training, save_model], validation_split=0.2)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_19 (LSTM)               (None, 50, 64)            433664    
_________________________________________________________________
lstm_20 (LSTM)               (None, 128)               98816     
_________________________________________________________________
dense_38 (Dense)             (None, 124)               15996     
_________________________________________________________________
dense_39 (Dense)             (None, 64)                8000      
_________________________________________________________________
dense_40 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_41 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_42 (Dense)             (None, 8)                

KeyboardInterrupt: ignored

In [ ]:
model_complied.summary() # adam default

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_17 (LSTM)               (None, 50, 64)            433664    
_________________________________________________________________
lstm_18 (LSTM)               (None, 128)               98816     
_________________________________________________________________
dense_33 (Dense)             (None, 124)               15996     
_________________________________________________________________
dense_34 (Dense)             (None, 64)                8000      
_________________________________________________________________
dense_35 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_36 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_37 (Dense)             (None, 8)                

In [ ]:
KT.start_training(X, Y, 1000, callbacks=[stop_training, save_model])

Epoch 1/1000
63/63 [==============================] - 25s 181ms/step - loss: 53.8892 - accuracy: 0.1380
INFO:tensorflow:Assets written to: /content/drive/MyDrive/MODEL/Model_300/assets
Epoch 2/1000
63/63 [==============================] - 11s 180ms/step - loss: 7.3827 - accuracy: 0.1251
Epoch 3/1000
63/63 [==============================] - 11s 181ms/step - loss: 7.6546 - accuracy: 0.1316
Epoch 4/1000
63/63 [==============================] - 11s 181ms/step - loss: 6.2368 - accuracy: 0.1395
INFO:tensorflow:Assets written to: /content/drive/MyDrive/MODEL/Model_300/assets
Epoch 5/1000
63/63 [==============================] - 15s 232ms/step - loss: 2.0795 - accuracy: 0.1495
INFO:tensorflow:Assets written to: /content/drive/MyDrive/MODEL/Model_300/assets
Epoch 6/1000
63/63 [==============================] - 11s 182ms/step - loss: 2.0773 - accuracy: 0.1495
Epoch 7/1000
63/63 [==============================] - 11s 180ms/step - loss: 2.0750 - accuracy: 0.1495
Epoch 8/1000
63/63 [===============

KeyboardInterrupt: ignored